# 14. 作用域（Scope）

作用域决定变量名在哪里可见。理解 LEGB、global/nonlocal、闭包与晚绑定，能避免大量“找不到变量/改不到变量”的 bug。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 第 13 节：函数基础


## 知识点地图

- 1. LEGB：名字查找规则
- 2. global：在函数里绑定全局变量
- 3. nonlocal：修改外层函数变量（闭包状态）
- 4. 闭包：函数携带外层状态
- 5. 晚绑定：循环里创建 lambda
- 6. globals()/locals()（了解）：查看当前命名空间


## 自检清单（学完打勾）

- [ ] 理解 LEGB：Local/Enclosing/Global/Built-in
- [ ] 理解 global 修改全局变量的含义与风险
- [ ] 掌握 nonlocal 修改外层函数变量
- [ ] 理解闭包保存状态
- [ ] 理解循环里 lambda 的晚绑定问题与修复


## 知识点 1：LEGB：名字查找规则

名字查找顺序：Local -> Enclosing -> Global -> Built-in。


In [ ]:
x = 'global'

def outer():
    x = 'enclosing'
    def inner():
        x = 'local'
        return x
    return inner(), x

print(outer())
print(x)


## 知识点 2：global：在函数里绑定全局变量

global 会让赋值指向全局名字。慎用：会让函数变得难测试、难复用。


In [ ]:
count = 0

def inc_global():
    global count
    count += 1

inc_global(); inc_global()
print(count)


## 知识点 3：nonlocal：修改外层函数变量（闭包状态）

nonlocal 用于修改“外层函数作用域”的变量，是实现计数器/缓存的常见手段。


In [ ]:
def make_counter():
    n = 0
    def inc():
        nonlocal n
        n += 1
        return n
    return inc

c = make_counter()
print(c(), c(), c())


## 知识点 4：闭包：函数携带外层状态

返回的函数会“记住”外层变量的绑定，从而携带状态。


In [ ]:
def make_adder(k):
    def add(x):
        return x + k
    return add

add10 = make_adder(10)
print(add10(5))


## 知识点 5：晚绑定：循环里创建 lambda

lambda 捕获的是变量本身（同一个 i），循环结束后 i 变成最后值，所以结果一样。


In [ ]:
funcs = []
for i in range(3):
    funcs.append(lambda: i)
print([f() for f in funcs])

funcs = []
for i in range(3):
    funcs.append(lambda i=i: i)  # 把当前值绑定到默认参数
print([f() for f in funcs])


## 知识点 6：globals()/locals()（了解）：查看当前命名空间

globals/locals 返回字典视图，可用于调试；不建议用它们写“魔法代码”。


In [ ]:
print('x' in globals())


## 常见坑

- 滥用 global 会让代码难维护
- 循环里 lambda 常见晚绑定坑（用默认参数或 functools.partial 修复）


## 综合小案例：实现带状态的计数器（闭包）

实现 counter(start=0)：返回函数 next()，每次调用返回递增值。


In [ ]:
def counter(start=0):
    n = start
    def next_():
        nonlocal n
        n += 1
        return n
    return next_

c = counter(10)
print(c(), c(), c())


## 自测题（不写代码也能回答）

- LEGB 分别代表什么？
- nonlocal 与 global 的区别是什么？
- 晚绑定为什么会导致 [2,2,2]？


## 练习题（建议写代码）

- 写 make_power(exp)：返回函数 f(x)=x**exp。
- 写一个例子：在函数里不写 global 就给全局变量赋值会发生什么？
